In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/myspeakerrecognitiondata/archive/16000_pcm_speeches/tf_Wav_reader.py
/kaggle/input/myspeakerrecognitiondata/archive/16000_pcm_speeches/other/exercise_bike.wav
/kaggle/input/myspeakerrecognitiondata/archive/16000_pcm_speeches/other/pink_noise.wav
/kaggle/input/myspeakerrecognitiondata/archive/16000_pcm_speeches/_background_noise_/10convert.com_Audience-Claps_daSG5fwdA7o.wav
/kaggle/input/myspeakerrecognitiondata/archive/16000_pcm_speeches/_background_noise_/running_tap.wav
/kaggle/input/myspeakerrecognitiondata/archive/16000_pcm_speeches/_background_noise_/doing_the_dishes.wav
/kaggle/input/myspeakerrecognitiondata/archive/16000_pcm_speeches/_background_noise_/dude_miaowing.wav
/kaggle/input/myspeakerrecognitiondata/archive/16000_pcm_speeches/Arunanshu/611.wav
/kaggle/input/myspeakerrecognitiondata/archive/16000_pcm_speeches/Arunanshu/3845.wav
/kaggle/input/myspeakerrecognitiondata/archive/16000_pcm_speeches/Arunanshu/1591.wav
/kaggle/input/myspeakerrecognitiondata/archive

In [2]:
import tensorflow as tf
import os
from os.path import isfile, join
import numpy as np
import shutil
from tensorflow import keras
from pathlib import Path
from IPython.display import display, Audio
import subprocess

In [3]:
!cp -r "../input/myspeakerrecognitiondata/archive" ./

In [4]:
data_directory = "./archive/16000_pcm_speeches"
audio_folder = "audio"
noise_folder = "noise"

audio_path = os.path.join(data_directory, audio_folder)
noise_path = os.path.join(data_directory, noise_folder)

In [5]:
audio_path

'./archive/16000_pcm_speeches/audio'

In [6]:
valid_split = 0.1

shuffle_seed = 43

sample_rate = 16000

scale = 0.5

batch_size = 128

epochs = 15

In [7]:
for folder in os.listdir(data_directory):
    if os.path.isdir(os.path.join(data_directory, folder)):
        if folder in [audio_folder, noise_folder]:
            
            continue
        elif folder in ["other", "_background_noise_"]:
            
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(noise_path, folder),
            )
        else:
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(audio_path, folder),
            )

In [8]:
os.listdir(noise_path)

['other', '_background_noise_']

In [9]:
noise_paths = []
for subdir in os.listdir(noise_path):
    subdir_path = Path(noise_path) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

In [10]:
noise_paths

['archive/16000_pcm_speeches/noise/other/exercise_bike.wav',
 'archive/16000_pcm_speeches/noise/other/pink_noise.wav',
 'archive/16000_pcm_speeches/noise/_background_noise_/dude_miaowing.wav',
 'archive/16000_pcm_speeches/noise/_background_noise_/running_tap.wav',
 'archive/16000_pcm_speeches/noise/_background_noise_/doing_the_dishes.wav',
 'archive/16000_pcm_speeches/noise/_background_noise_/10convert.com_Audience-Claps_daSG5fwdA7o.wav']

In [11]:
command = (
    "for dir in `ls -1 " + noise_path + "`; do "
    "for file in `ls -1 " + noise_path + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)

In [12]:
os.system(command)
def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate == sample_rate:
        slices = int(sample.shape[0] / sample_rate)
        sample = tf.split(sample[: slices * sample_rate], slices)
        return sample
    else:
        print("Sampling rate for",path, "is incorrect")
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

In [13]:
def paths_and_labels_to_dataset(audio_paths, labels):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))

In [14]:
def path_to_audio(path):
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, sample_rate)
    return audio

In [15]:
def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)

        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)

        audio = audio + noise * prop * scale

    return audio

In [16]:
def audio_to_fft(audio):
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    return tf.math.abs(fft[:, : (audio.shape[1] ), :])

In [17]:

class_names = os.listdir(audio_path)
print(class_names,)

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Speaker:",(name))
    dir_path = Path(audio_path) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

['Hari', 'Arunanshu', 'Dhanush', 'Pratham']
Speaker: Hari
Speaker: Arunanshu
Speaker: Dhanush
Speaker: Pratham


In [18]:
# Shuffle to generate random data
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(audio_paths)
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(labels)

In [19]:
# Split into training and validation
num_val_samples = int(valid_split * len(audio_paths))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]


valid_audio_paths = audio_paths[-num_val_samples:]
valid_labels = labels[-num_val_samples:]

In [20]:
# Create datasets, one for training and the other for validation
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=shuffle_seed).batch(32)

In [21]:
# Create datasets, one for training and the other for validation
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=shuffle_seed).batch(32)

In [22]:
train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 16000, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [23]:
from tensorflow.keras.layers import Conv1D

In [24]:
def residual_block(x, filters, conv_num = 3, activation = "relu"):
    s = keras.layers.Conv1D(filters, 1, padding = "same")(x)
    
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding = "same")(x)
        x = keras.layers.Activation(activation)(x)
    
    x = keras.layers.Conv1D(filters, 3, padding = "same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    
    return keras.layers.MaxPool1D(pool_size = 2, strides = 2)(x)

def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape = input_shape, name = "input")
    
    x = residual_block(inputs, 16, 2)
    x = residual_block(inputs, 32, 2)
    x = residual_block(inputs, 64, 3)
    x = residual_block(inputs, 128, 3)
    x = residual_block(inputs, 128, 3)
    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    
    outputs = keras.layers.Dense(num_classes, activation = "softmax", name = "output")(x)
    
    return keras.models.Model(inputs = inputs, outputs = outputs)

model = build_model((sample_rate , 1), len(class_names))

model.summary()

model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) 

model_save_filename = "model.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(model_save_filename, monitor="val_accuracy", save_best_only=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 16000, 1)]           0         []                            
                                                                                                  
 conv1d_15 (Conv1D)          (None, 16000, 128)           512       ['input[0][0]']               
                                                                                                  
 activation_10 (Activation)  (None, 16000, 128)           0         ['conv1d_15[0][0]']           
                                                                                                  
 conv1d_16 (Conv1D)          (None, 16000, 128)           49280     ['activation_10[0][0]']       
                                                                                              

In [25]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/15
124/124 [==============================] - ETA: 0s - loss: 0.8453 - accuracy: 0.6116 

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


124/124 [==============================] - 1800s 15s/step - loss: 0.8453 - accuracy: 0.6116 - val_loss: 0.7173 - val_accuracy: 0.6659
Epoch 2/15
124/124 [==============================] - 1882s 15s/step - loss: 0.6858 - accuracy: 0.6812 - val_loss: 0.7583 - val_accuracy: 0.6500
Epoch 3/15
124/124 [==============================] - 1910s 15s/step - loss: 0.5594 - accuracy: 0.7410 - val_loss: 0.6700 - val_accuracy: 0.7097
Epoch 4/15
124/124 [==============================] - 1937s 16s/step - loss: 0.4279 - accuracy: 0.8042 - val_loss: 0.6021 - val_accuracy: 0.7545
Epoch 5/15
124/124 [==============================] - 1891s 15s/step - loss: 0.3996 - accuracy: 0.8233 - val_loss: 0.8425 - val_accuracy: 0.7182
Epoch 6/15
124/124 [==============================] - 1898s 15s/step - loss: 0.2703 - accuracy: 0.8821 - val_loss: 0.7466 - val_accuracy: 0.7886
Epoch 7/15
124/124 [==============================] - 1899s 15s/step - loss: 0.1464 - accuracy: 0.9425 - val_loss: 0.5836 - val_accuracy: 0.8

In [26]:
print("Accuracy of model:",model.evaluate(valid_ds))

55/55 [==============================] - 65s 1s/step - loss: 0.7642 - accuracy: 0.8591
Accuracy of model: [0.7641701698303223, 0.8590909242630005]


In [27]:
SAMPLES_TO_DISPLAY = 10

test_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds = test_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

test_ds = test_ds.map(lambda x, y: (add_noise(x, noises, scale=scale), y))

for audios, labels in test_ds.take(1):
    ffts = audio_to_fft(audios)
    y_pred = model.predict(ffts)
    rnd = np.random.randint(0, batch_size, SAMPLES_TO_DISPLAY)
    audios = audios.numpy()[rnd, :, :]
    labels = labels.numpy()[rnd]
    y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(SAMPLES_TO_DISPLAY):
        print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[labels[index]],
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[y_pred[index]],
            )
        )
        if labels[index] ==y_pred[index]:
            print("Welcome")
        else:
            print("Sorry")
        print("The speaker is" if labels[index] == y_pred[index] else "", class_names[y_pred[index]])

4/4 [==============================] - 5s 1s/step
Speaker: Dhanush	Predicted: Dhanush
Welcome
The speaker is Dhanush
Speaker: Hari	Predicted: Dhanush
Sorry
 Dhanush
Speaker: Dhanush	Predicted: Dhanush
Welcome
The speaker is Dhanush
Speaker: Dhanush	Predicted: Dhanush
Welcome
The speaker is Dhanush
Speaker: Hari	Predicted: Pratham
Sorry
 Pratham
Speaker: Pratham	Predicted: Dhanush
Sorry
 Dhanush
Speaker: Dhanush	Predicted: Dhanush
Welcome
The speaker is Dhanush
Speaker: Pratham	Predicted: Dhanush
Sorry
 Dhanush
Speaker: Arunanshu	Predicted: Dhanush
Sorry
 Dhanush
Speaker: Dhanush	Predicted: Dhanush
Welcome
The speaker is Dhanush


In [28]:
 def paths_to_dataset(audio_paths):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    return tf.data.Dataset.zip((path_ds))

def predict(path, labels):
    test = paths_and_labels_to_dataset(path, labels)


    test = test.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
    )
    test = test.prefetch(tf.data.experimental.AUTOTUNE)


    test = test.map(lambda x, y: (add_noise(x, noises, scale=scale), y))

    for audios, labels in test.take(1):
        ffts = audio_to_fft(audios)
        y_pred = model.predict(ffts)
        rnd = np.random.randint(0, 1, 1)
        audios = audios.numpy()[rnd, :]
        labels = labels.numpy()[rnd]
        y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(1):
            print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
            "[92m",y_pred[index],
                "[92m", y_pred[index]
                )
            )
            
            print("Speaker Predicted:",class_names[y_pred[index]])



In [29]:
path = ["../input/myspeakerrecognitiondata/archive/16000_pcm_speeches/Dhanush/1013.wav"]
labels = ["unknown"]
try:
    predict(path, labels)
except:
    print("Error! Check if the file correctly passed or not!")

1/1 [==============================] - 0s 251ms/step
Speaker: 2	Predicted: 2
Speaker Predicted: Dhanush
